<sub><sup>Copyright (C) 2025 J. Heinzmann, A. Benady, ETH Zürich</sub></sup>

<sub><sup>
This file is part of the `nlfea` package for the course Computational Mechanics II: Nonlinear FEA at ETH Zurich.
This software is exclusively intended for personal, educational purposes only in the context of the above course.
It must not be used outside of this scope without the explicit consent of the author(s).
</sub></sup>

# Project 3

The third project revolves around the FE implementation of non-linear continuum mechanics and hyperelasticity.

This notebook will guide you through all the necessary steps.

## 1. Implementation of non-linear continuum mechanics

We start with the implementation of continuum elements with large deformations.
For this, we'll use a simple rectangular block under tension with a St. Venant Kirchhoff material model for which we create the setup below.

In [ ]:
# external dependencies
import numpy as np

# internal dependencies
from nlfea import Model, Solver, mesh, materials, elements, solution, post

# initialize empty model object
model_SVK = Model()

# mesh
W = 10
L = 100
(model_SVK.nodes, model_SVK.connectivity) = mesh.plane.rectangle(
    length_x=W, length_y=L, num_nodes_x=10, num_nodes_y=50, element_type="quad4"
)

# define material model
model_SVK.material = materials.StVenantKirchhoff(λ=121, μ=80)

# define type of element
model_SVK.parent_element = elements.Quad(
    kinematics="nonlinear", element_type="quad4", integration="full"
)


# find node sets for the application of BCs
bottom_edge = mesh.nodesets.find(model_SVK.nodes, Y=0)
bottom_leftcorner = mesh.nodesets.find(model_SVK.nodes, X=0, Y=0)
top_edge = mesh.nodesets.find(model_SVK.nodes, Y=L)

# Dirichlet BCs
model_SVK.dirichlet_bcs = (("fix_X", bottom_leftcorner), ("fix_Y", bottom_edge))

# Neumann BCs
t_bar = 100
model_SVK.neumann_bcs = (("t_Y", top_edge, t_bar),)

# solver parameters
solver_SVK = Solver()
solver_SVK.num_increments = 10
solver_SVK.tolerance_nr = 1e-7
solver_SVK.max_iter = 20
solver_SVK.control_method = "load"

# get DOF used for the monitoring in the plots
monitoring_node = mesh.nodesets.find(model_SVK.nodes, X=W, Y=L)
monitoring_node_dof = np.array([[monitoring_node[0], 1]])
solver_SVK.monitoring_dof = int(
    monitoring_node_dof[0, 0] * 2 + monitoring_node_dof[0, 1]
)

# set output directory
solver_SVK.result_dir = "project3_SVK"

**TASK 1**: Add the missing definitions for the basis functions in `nlfea/elements/shape_functions.py`.

**TASK 2**: Add the missing definitions for the bilinear quadrilateral element in `nlfea/elements/quad.py`.

Now, we can continue with the incremental-iterative solution process.

In [ ]:
# preprocess model and start computations
model_SVK.prepare()
solution.incremental.solve(model_SVK, solver_SVK)

Next, want to visualize the residuals as well as the force-displacement result for this test.

In [ ]:
# plot residuals
post.plots.residuals(solver_SVK.monitoring)

# plot results
post.plots.force_displacement(solver_SVK.monitoring)

Finally, we want to visualize the field results, namely the displacement and the pushed-forward Cauchy stress $\sigma$. Note that for the latter, we need to project the values from the Gauss points to the nodes for visualization (this is common in FE programs), which -- if you're curious -- you can find in `nlfea/post/gausspoint_to_nodes.py`. With the chosen BCs, you should see a homogeneous stress state (up to numerical round-off errors).

In [ ]:
post.interactive.continuum(solver_SVK.result_dir, field="u")
post.interactive.continuum(solver_SVK.result_dir, field="σ")

## 2. Implementation of hyperelastic material laws

Next, we devote ourselves to the implementation of hyperelastic material laws.
While before, we've been using the St. Venant-Kirchhoff material model, we now want to use a neo-Hookean material.

**TASK 3:** Implement the missing definitions of the Mooney-Rivlin material model in `nlfea/materials/neo_hooke.py`.

With your implementation, run the same test setup.

In [ ]:
# initialize empty model object
model_NH = Model()

# mesh
W = 10
L = 100
(model_NH.nodes, model_NH.connectivity) = mesh.plane.rectangle(
    length_x=W, length_y=L, num_nodes_x=10, num_nodes_y=50, element_type="quad4"
)

# define material model
model_NH.material = materials.NeoHooke(λ=121, μ=80)

# define type of element
model_NH.parent_element = elements.Quad(
    kinematics="nonlinear", element_type="quad4", integration="full"
)


# find node sets for the application of BCs
bottom_edge = mesh.nodesets.find(model_NH.nodes, Y=0)
bottom_leftcorner = mesh.nodesets.find(model_NH.nodes, X=0, Y=0)
top_edge = mesh.nodesets.find(model_NH.nodes, Y=L)

# Dirichlet BCs
model_NH.dirichlet_bcs = (("fix_X", bottom_leftcorner), ("fix_Y", bottom_edge))

# Neumann BCs
t_bar = 100
model_NH.neumann_bcs = (("t_Y", top_edge, t_bar),)

# solver parameters
solver_NH = Solver()
solver_NH.num_increments = 10
solver_NH.tolerance_nr = 1e-7
solver_NH.max_iter = 20
solver_NH.control_method = "load"

# get DOF used for the monitoring in the plots
monitoring_node = mesh.nodesets.find(model_NH.nodes, X=W, Y=L)
monitoring_node_dof = np.array([[monitoring_node[0], 1]])
solver_NH.monitoring_dof = int(
    monitoring_node_dof[0, 0] * 2 + monitoring_node_dof[0, 1]
)

# set output directory
solver_NH.result_dir = "project3_NH"

# preprocess model and start computations
model_NH.prepare()
solution.incremental.solve(model_NH, solver_NH)

# plot residuals
post.plots.residuals(solver_NH.monitoring)

# plot results
post.plots.force_displacement(solver_NH.monitoring)

Finally, we again want to visualize the field results, namely the displacement and the pushed-forward Cauchy stress $\sigma$.

In [ ]:
post.interactive.continuum(solver_NH.result_dir, field="u")
post.interactive.continuum(solver_NH.result_dir, field="σ")

## 3. Application

Finally, to test your implementation with a more interesting test setup, you can run a common test called 'Cooke's membrane', where a traction load is applied on the right edge.

In [ ]:
# initialize empty model object
model = Model()

# mesh
(model.nodes, model.connectivity) = mesh.plane.cooksmembrane(
    num_nodes_x=25, num_nodes_y=25, element_type="quad4"
)

# define type of element
model.parent_element = elements.Quad(
    kinematics="nonlinear", element_type="quad4", integration="full"
)

# material
model.material = materials.NeoHooke(λ=121, μ=40)

# find node sets for the application of BCs
left_edge = mesh.nodesets.find(model.nodes, X=0)
right_edge = mesh.nodesets.find(model.nodes, X=max(model.nodes[:, 0]))

# Dirichlet BCs
model.dirichlet_bcs = (("fix_X", left_edge), ("fix_Y", left_edge))

# Neumann BCs
model.neumann_bcs = (("t_Y", right_edge, 5),)

# solver parameters
solver = Solver()
solver.num_increments = 10
solver.tolerance_nr = 1e-7
solver.max_iter = 20
solver.control_method = "load"

# get DOF used for the monitoring in the plots
monitoring_node = mesh.nodesets.find(
    model.nodes, X=max(model.nodes[:, 0]), Y=max(model.nodes[:, 1])
)
monitoring_node_dof = np.array([[monitoring_node[0], 0]])
solver.monitoring_dof = int(monitoring_node_dof[0, 0] * 2 + monitoring_node_dof[0, 1])

# set output directory
solver.result_dir = "misc_cooksmembrane"

# preprocess model and start computations
model.prepare()
solution.incremental.solve(model, solver)

# plot residuals
post.plots.residuals(solver.monitoring)

# plot deformed mesh
post.interactive.continuum(solver.result_dir)


For the submission of this project, upload on Moodle:

- this Jupyter notebook,
- the file `elements/shape_functions.py`,
- the file `elements/quad.py`, and
- the file `materials/neo_hooke.py`.